In [28]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dropout, Flatten, Dense, Conv1D, Reshape, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
data = pd.read_csv(r"C:\Users\Yash Vikram Singh\Downloads\emotions.csv\emotions.csv")

In [3]:
import chardet

with open(r"C:\Users\Yash Vikram Singh\Downloads\emotions.csv\emotions.csv", "rb") as file:
    result = chardet.detect(file.read())
    print(result["encoding"])

ascii


In [4]:
# import kagglehub
# kagglehub.dataset_download('birdy654/eeg-brainwave-dataset-feeling-emotions', path='emotions.csv')

In [5]:
print(data.columns)

Index(['# mean_0_a', 'mean_1_a', 'mean_2_a', 'mean_3_a', 'mean_4_a',
       'mean_d_0_a', 'mean_d_1_a', 'mean_d_2_a', 'mean_d_3_a', 'mean_d_4_a',
       ...
       'fft_741_b', 'fft_742_b', 'fft_743_b', 'fft_744_b', 'fft_745_b',
       'fft_746_b', 'fft_747_b', 'fft_748_b', 'fft_749_b', 'label'],
      dtype='object', length=2549)


In [6]:
label_mappping = {'NEGATIVE':0, 'NEUTRAL':1, 'POSITIVE':2}
data['label'] = data['label'].map(label_mappping)

In [7]:
data.head()

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,...,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,0
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,...,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,1
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,...,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,2
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,...,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,2
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,...,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,1


In [8]:
X = data.iloc[:, :-1].values
y = pd.get_dummies(data['label']).values


In [9]:
X_train, X_val, Y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=11)

In [10]:
X_train_lstm = np.expand_dims(X_train, axis=-1)
X_val_lstm = np.expand_dims(X_val, axis=-1)

In [11]:
i_lstm = Input(shape=(X_train.shape[1], 1))
x_lstm = LSTM(256, return_sequences=True)(i_lstm)
x_lstm = Dropout (0.6)(x_lstm)
x_lstm = Flatten()(x_lstm)
y_lstm = Dense(3, activation='softmax')(x_lstm)
model_lstm = Model(i_lstm, y_lstm)

In [12]:
adam_lstm = Adam(learning_rate=0.000001)
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience = 20, restore_best_weights=True)

In [22]:
model_lstm.compile(optimizer=adam_lstm,loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
lstm_h = model_lstm.fit(X_train_lstm, Y_train,
                        batch_size=512,
                        validation_data=(X_val_lstm, y_val),
                        epochs=1,
                        callbacks=[es])

4/4 [==============================] - 497s 122s/step - loss: 1.0983 - accuracy: 0.4258 - val_loss: 0.9144 - val_accuracy: 0.6557


In [14]:
input_shape_capsule = (X_train.shape[1], 1)
input_layer_capsule = Input(shape=input_shape_capsule)

In [15]:
conv1_params={
    "filters": 256,
    "kernel_size": 11,
    "strides": 1,
    "padding":"valid",
    "activation": "softmax"
}

In [16]:
primary_caps = Conv1D(**conv1_params)(input_layer_capsule)
primary_caps_reshaped = Reshape((-1, 256))(primary_caps)

In [17]:
num_capsule = 5
dim_capsule = 16
capsule_layer = Dense(num_capsule * dim_capsule, activation = "relu")(primary_caps_reshaped)
capsule_layer_flattened = Flatten()(capsule_layer)

In [18]:
def squash(x, axis=-1):
    squared_norm = tf.reduce_sum(tf.square(x), axis, keepdims=True)
    scale = squared_norm / (1+squared_norm)
    return scale*x

In [19]:
capsule_layer_squashed = Lambda(squash)(capsule_layer_flattened)
output_layer_capsule = Dense(3, activation='softmax')(capsule_layer_squashed)

In [20]:
model_capsule = Model(inputs=input_layer_capsule, outputs=output_layer_capsule)
model_capsule.compile(optimizer="adam",
                    loss="categorical_crossentropy",
                    metrics=['accuracy'])

In [46]:
population_size=10
generations=5
mutation_rate=0.1

In [47]:
def create_population():
    return [(model_lstm.get_weights(), model_capsule.get_weights()) for _ in range(population_size)]


def fitness_function(weights):
    lstm_weights, capsule_weights = weights
    model_lstm.set_weights(lstm_weights)
    model_capsule.set_weights(capsule_weights)
    
    model_lstm.compile(optimizer=adam_lstm, loss='categorical_crossentropy', metrics=['accuracy'])
    model_capsule.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    
    _, accuracy_lstm = model_lstm.evaluate(X_val_lstm, y_val, verbose=0)
    _, accuracy_capsule = model_capsule.evaluate(X_val_lstm, y_val, verbose=0)
    
    return (accuracy_lstm + accuracy_capsule) / 2


def select_parents(population, fitnesses):
    idx=np.argsort(fitnesses)[-2:]
    return [population[i] for i in idx]

def crossover(parent1, parent2):
    lstm_child = []
    capsule_child = []
    
    for w1, w2 in zip(parent1[0], parent2[0]):
        split_point = np.random.randint(w1.size)
        w1_flat = w1.flatten()
        w2_flat = w2.flatten()
        child_flat = np.concatenate([w1_flat[:split_point], w2_flat[split_point:]])
        lstm_child.append(child_flat.reshape(w1.shape))
        
    for w1, w2 in zip(parent1[1], parent2[1]):
        split_point = np.random.randint(w1.size)
        w1_flat = w1.flatten()
        w2_flat = w2.flatten()
        child_flat = np.concatenate([w1_flat[:split_point], w2_flat[split_point:]])
        capsule_child.append(child_flat.reshape(w1.shape))
        
    return (lstm_child, capsule_child)

def mutate(weights):
    lstm_weights, capsule_weights = weights
    
    for w in lstm_weights:
        if np.random.rand() < mutation_rate:
            mutation = np.random.normal(0, 0.1, w.shape)
            w += mutation
            
    for w in capsule_weights:
        if np.random.rand() < mutation_rate:
            mutation = np.random.normal(0, 0.1, w.shape)
            w += mutation
            
    return (lstm_weights, capsule_weights)


In [24]:
population = create_population()

for generation in range(generations):
    fitnesses = [fitness_function(weights) for weights in population]
    parents = select_parents(population, fitnesses)
    new_population = []
    
    for _ in range(population_size // 2):
        child1 = crossover(parents[0], parents[1])
        child2 = crossover(parents[1], parents[0])
        new_population.extend([mutate(child1), mutate(child2)])
        
    population = new_population
    print(f"Generation{generation + 1}: Best Aveage Accuracy = {max(fitnesses)}")
    
best_weights = population[np.argmax(fitnesses)]
model_lstm.set_weights(best_weights[0])
model_capsule.set_weights(best_weights[1])


model_lstm.evaluate(X_val_lstm, y_val)
model_capsule.evaluate(X_val_lstm, y_val)

Generation1: Best Aveage Accuracy = 0.5913348942995071
Generation2: Best Aveage Accuracy = 0.649882897734642
Generation3: Best Aveage Accuracy = 0.6885245740413666
Generation4: Best Aveage Accuracy = 0.766978919506073
Generation5: Best Aveage Accuracy = 0.7587821781635284
14/14 [==============================] - 1s 101ms/step - loss: 1.0491 - accuracy: 0.5714


[1.0490883588790894, 0.5714285969734192]

In [27]:
def flatten_weights(model):
    """Flatten the weights of a Keras model into a 1D array."""
    flattened_weights = []
    for layer in model.layers:
        layer_weights = layer.get_weights()
        if layer_weights:
            flattened_weights.extend([w.flatten() for w in layer_weights])
    return np.concatenate(flattened_weights)

def reconstruct_weights(flattened_weights, model):
    start_idx = 0
    for layer in model.layers:
        layer_weights = layer.get_weights()
        if layer_weights: 
            weight_shapes = [w.shape for w in layer_weights]
            new_weights = [flattened_weights[start_idx:start_idx + np.prod(shape)].reshape(shape)
                           for shape in weight_shapes]
            layer.set_weights(new_weights)
            start_idx += sum([np.prod(shape) for shape in weight_shapes])

def initialize_particles_in_chunks(num_particles, weight_dim, chunk_size=100000):
    """Initialize particles and velocities in chunks to save memory."""
    particles = np.zeros((num_particles, weight_dim), dtype=np.float32)
    velocities = np.zeros((num_particles, weight_dim), dtype=np.float32)
    for start in range(0, weight_dim, chunk_size):
        end = min(start + chunk_size, weight_dim)
        particles[:, start:end] = np.random.uniform(-1, 1, (num_particles, end - start))
        velocities[:, start:end] = np.random.uniform(-0.1, 0.1, (num_particles, end - start))
    return particles, velocities

def fitness_function(weights, model, X_val, y_val):
    """Evaluate the fitness of the weights using validation loss."""
    reconstruct_weights(weights, model)
    loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
    return loss 

def optimize_with_pso_reduced_memory(model, X_train, Y_train, X_val, y_val, num_particles=5, max_iterations=20, w=0.5, c1=1.5, c2=1.5):
    model_clone = clone_model(model)
    model_clone.set_weights(model.get_weights())
    model_clone.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    initial_weights = flatten_weights(model_clone).astype(np.float32)
    weight_dim = len(initial_weights)

    particles, velocities = initialize_particles_in_chunks(num_particles, weight_dim)

    pbest = particles.copy()
    pbest_fitness = np.array([fitness_function(p, model_clone, X_val, y_val) for p in particles])
    gbest = particles[np.argmin(pbest_fitness)]
    gbest_fitness = min(pbest_fitness)

    for iteration in range(max_iterations):
        print(f"Iteration {iteration + 1}/{max_iterations} - Best Fitness: {gbest_fitness}")

        for i in range(num_particles):
            r1, r2 = np.random.rand(weight_dim).astype(np.float32), np.random.rand(weight_dim).astype(np.float32)
            velocities[i] = (w * velocities[i] +
                             c1 * r1 * (pbest[i] - particles[i]) +
                             c2 * r2 * (gbest - particles[i]))

            particles[i] += velocities[i]
            fitness = fitness_function(particles[i], model_clone, X_val, y_val)

            if fitness < pbest_fitness[i]:
                pbest[i] = particles[i]
                pbest_fitness[i] = fitness
                if fitness < gbest_fitness:
                    gbest = particles[i]
                    gbest_fitness = fitness

    reconstruct_weights(gbest, model)
    return model

optimized_lstm = optimize_with_pso_reduced_memory(model_lstm, X_train_lstm, Y_train, X_val_lstm, y_val)

lstm_loss, lstm_accuracy = optimized_lstm.evaluate(X_val_lstm, y_val, verbose=1)
print(f"Optimized LSTM - Loss: {lstm_loss}, Accuracy: {lstm_accuracy}")




Iteration 1/20 - Best Fitness: 135.7471160888672
Iteration 2/20 - Best Fitness: 129.58206176757812
Iteration 3/20 - Best Fitness: 120.79552459716797
Iteration 4/20 - Best Fitness: 102.43396759033203
Iteration 5/20 - Best Fitness: 102.43396759033203
Iteration 6/20 - Best Fitness: 87.8499526977539
Iteration 7/20 - Best Fitness: 84.42826843261719
Iteration 8/20 - Best Fitness: 82.93009185791016
Iteration 9/20 - Best Fitness: 79.98644256591797
Iteration 10/20 - Best Fitness: 77.41421508789062
Iteration 11/20 - Best Fitness: 71.90027618408203
Iteration 12/20 - Best Fitness: 68.6474380493164
Iteration 13/20 - Best Fitness: 68.6474380493164
Iteration 14/20 - Best Fitness: 68.6474380493164
Iteration 15/20 - Best Fitness: 68.6474380493164
Iteration 16/20 - Best Fitness: 68.16732025146484
Iteration 17/20 - Best Fitness: 64.87919616699219
Iteration 18/20 - Best Fitness: 64.87919616699219
Iteration 19/20 - Best Fitness: 64.87919616699219
Iteration 20/20 - Best Fitness: 64.87919616699219
14/14 [===